#

# This regression model is set to predict at what price you should list your item to get the most sales.

In [91]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
import sys, numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from numpy import argmax

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

df= pd.read_csv('https://raw.githubusercontent.com/ReneMazuela/Datasets/main/ebay_ecom.csv')

print(df.columns)

Index(['all_price', 'brand', 'item_name', 'rating', 'comments',
       'shipping_cost', 'shipping_from', 'sold_quantity', 'view_quantity'],
      dtype='object')


In [121]:
df.columns = df.columns.str.strip()
df = df.replace('None', 0)
df = df.replace(['shipping', 'Free','Shipping'], 0)
df = df.replace(',', '', regex=True)
df = df.dropna()

In [94]:
def convert_price(price_str):
    # If price_str is not a string, return it as is
    if not isinstance(price_str, str):
        return price_str

    # Remove commas
    price_str = price_str.replace(',', '')

    if "to" in price_str:
        prices = price_str.split(" to ")
        lower = float(prices[0])
        upper = float(prices[1])
        return (lower + upper) / 2
    else:
        return float(price_str)

# Apply the function to your data
df['all_price'] = df['all_price'].apply(convert_price)


In [95]:
from sklearn.preprocessing import LabelEncoder

# Initialize a new LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to each of the specified columns
df['shipping_from'] = le.fit_transform(df['shipping_from'])
df['brand'] = le.fit_transform(df['brand'])
df['comments'] = le.fit_transform(df['comments'])

In [96]:
df.sample(5)

,all_price,brand,item_name,rating,comments,shipping_cost,shipping_from,sold_quantity,view_quantity
849,204.99,185,Samsung Galaxy S21 5G G991U 128/256GB Unlocked...,2,7,29.69,22,0,0
1195,139.00,185,Samsung Galaxy Z Flip3 5G SM-F711U - 128GB - (...,0,7,30.55,22,0,0
1228,129.95,17,4G LTE Kyocera DuraXV Extreme E4810 16GB Veriz...,6,5,28.17,22,7,9
1794,234.99,32,Apple iPhone 11 64GB - T-Mobile / Metro PCS - ...,6,7,30.17,22,0,0
408,144.95,185,Samsung Galaxy A71 5G SM-A716U1 - 128GB - Pris...,0,5,29.31,22,0,0


In [97]:
df.drop(['item_name'], axis=1, inplace=True)

In [122]:

df.sample(1000)

,all_price,brand,rating,comments,shipping_cost,shipping_from,sold_quantity,view_quantity
64,609.380,133,5,0,34.63,22,18,0
1677,699.990,220,0,5,28.5,22,0,0
1867,121.490,32,0,7,0,2,0,0
816,217.490,179,101,7,29.12,22,0,0
699,90.500,90,0,7,25.87,22,0,0
...,...,...,...,...,...,...,...,...
1130,7.500,32,0,0,33.3,22,0,0
58,621.605,32,11,3,57.57,21,37,0
1646,305.850,47,0,0,52.18,22,0,0
791,386.990,32,174,2,33.06,22,435,26


In [112]:
df.isnull().sum()

all_price        0
brand            0
rating           0
comments         0
shipping_cost    0
shipping_from    0
sold_quantity    0
view_quantity    0
dtype: int64

In [113]:
df.fillna(df.mean(), inplace=True)

In [123]:
scaler = MinMaxScaler()

X = df.drop('all_price', axis=1)
y = df['all_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [126]:
from tensorflow.keras import backend as K
# Custom loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(loss=rmse, optimizer=Adam(), metrics=[rmse])

model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
48/48 [==============================] - 1s 2ms/step - loss: 286.5353 - rmse: 286.5353
Epoch 2/100
48/48 [==============================] - 0s 2ms/step - loss: 246.0633 - rmse: 246.0633
Epoch 3/100
48/48 [==============================] - 0s 3ms/step - loss: 205.1977 - rmse: 205.1977
Epoch 4/100
48/48 [==============================] - 0s 2ms/step - loss: 202.9065 - rmse: 202.9065
Epoch 5/100
48/48 [==============================] - 0s 2ms/step - loss: 203.2314 - rmse: 203.2314
Epoch 6/100
48/48 [==============================] - 0s 2ms/step - loss: 202.7238 - rmse: 202.7238
Epoch 7/100
48/48 [==============================] - 0s 2ms/step - loss: 203.1897 - rmse: 203.1897
Epoch 8/100
48/48 [==============================] - 0s 2ms/step - loss: 200.5063 - rmse: 200.5063
Epoch 9/100
48/48 [==============================] - 0s 2ms/step - loss: 201.4210 - rmse: 201.4210
Epoch 10/100
48/48 [==============================] - 0s 2ms/step - loss: 202.5538 - rmse: 202.5538
Epoch 11/

In [129]:
predictions = model.predict(X_test)

# Check the shapes of y_test and predictions
print("y_test shape:", y_test.shape)
print("predictions shape:", predictions.shape)

# Flatten both y_test and predictions if necessary
if len(y_test.shape) > 1:
    y_test = y_test.flatten()

if len(predictions.shape) > 1:
    predictions = predictions.flatten()

# Calculate RMSE
rmse_value = rmse(y_test, predictions)
print(f'RMSE on the test set: {rmse_value.numpy()}')

12/12 [==============================] - 0s 2ms/step
y_test shape: (384,)
predictions shape: (384, 1)
RMSE on the test set: 174.71899834861745


In [130]:
# Calculate mean absolute error (MAE)
mae = tf.keras.losses.MeanAbsoluteError()
mae_value = mae(y_test, predictions).numpy()

# Print RMSE and MAE
print(f"RMSE on the test set: {rmse_value:.2f}")
print(f"Mean Absolute Error (MAE) on the test set: {mae_value:.2f}")

RMSE on the test set: 174.72
Mean Absolute Error (MAE) on the test set: 126.95


In [134]:

new_data = np.array([[32,	5,	0,	34.63,	22,	19,	0]])

# Preprocess the new data point
scaled_new_data = scaler.transform(new_data)

# Make a prediction using the preprocessed new data point
new_prediction = model.predict(scaled_new_data)

# Print the prediction
print(f"Predicted value for the new data point: {new_prediction[0][0]:.2f}")

1/1 [==============================] - 0s 21ms/step
Predicted value for the new data point: 254.96
